In [352]:
import polars as pl
from scipy.stats import t
from pathlib import Path
import numpy as np
from desdeo.tools.indicators_unary import hv


def hv_from_front(front, ref_component=1.0):
    if len(front) == 0:
        return None

    pts = np.array([[p["f_norm"], p["c_norm"]] for p in front], dtype=float)

    return hv(pts, ref_component)


# load run
problem_name = "branin"
mode = "ranking"
n_gen = 100
n_runs = 1_000
ct = "5p0"
pop_size = 6

data_root = Path()
file_ext = "parquet"
results_path = Path("results/")
file_name = Path(
    data_root / results_path / f"{problem_name}_{mode}_gen{n_gen}_runs{n_runs}_ct{ct}_psize{pop_size}.{file_ext}"
)

df = pl.read_parquet(file_name)

# load pareto sets
n_gen_front = 10_000
psize_front = 1_000

fronts_path = Path("results/fronts/")
front_file_name = Path(fronts_path / f"{problem_name}_gen{n_gen_front}_psize{psize_front}.{file_ext}")

df_front = pl.read_parquet(front_file_name)

df

x_1,x_2,f_1,f_1_min,c_1,generation,run
f64,f64,f64,f64,f64,i32,i32
8.06511,3.902804,-126.891561,-126.891561,8.444131,1,0
-4.646495,0.75658,-417.394825,-417.394825,242.379461,1,0
7.264557,12.560944,-13.431644,-13.431644,138.147231,1,0
0.597151,10.534839,-108.351235,-108.351235,42.524934,1,0
4.679716,8.874035,-65.832868,-65.832868,60.830281,1,0
…,…,…,…,…,…,…
-3.870065,10.497932,-212.647314,-212.647314,10.768464,100,999
-3.239412,10.579177,-194.825699,-194.825699,-0.823038,100,999
-3.474125,9.240304,-214.726151,-214.726151,10.732061,100,999


## Summary statistics

In [371]:
# filter feasible solutions
per_run_gen = df.group_by(["generation", "run"]).agg(
    pl.when(pl.col("c_1") <= 0.0).then(pl.col("f_1_min")).otherwise(None).min().alias("gen_best_feasible")
)

# take cumulative minimum over all generations in each run
per_run_best_so_far = (
    per_run_gen.sort(["run", "generation"])
    .with_columns(pl.col("gen_best_feasible").fill_null(1_000_000).cum_min().over("run").alias("run_best_so_far_raw"))
    .with_columns(
        pl.when(pl.col("run_best_so_far_raw") == 1_000_000)
        .then(None)
        .otherwise(pl.col("run_best_so_far_raw"))
        .alias("run_best_so_far")
    )
).drop("run_best_so_far_raw")

# compute standard error
summary = (
    per_run_best_so_far.group_by("generation")
    .agg(
        pl.col("run_best_so_far").mean().alias("run_best_so_far_mean"),
        pl.col("run_best_so_far").std().alias("run_best_so_far_std"),
        pl.col("run_best_so_far").count().alias("best_n_feasible_runs"),
    )
    .with_columns(
        (pl.col("run_best_so_far_std") / pl.col("best_n_feasible_runs").sqrt()).alias("run_best_so_far_stderr")
    )
).sort("generation")

# compute 95% confidence intervals
summary = summary.with_columns(
    pl.when(pl.col("best_n_feasible_runs") > 1)
    .then(pl.Series("best_t_crit", t.ppf(0.975, summary["best_n_feasible_runs"] - 1)))
    .otherwise(None)
)

summary = summary.with_columns(
    (pl.col("run_best_so_far_mean") + pl.col("best_t_crit") * pl.col("run_best_so_far_stderr")).alias("best_ci_upper"),
    (pl.col("run_best_so_far_mean") - pl.col("best_t_crit") * pl.col("run_best_so_far_stderr")).alias("best_ci_lower"),
).sort("generation")

summary

generation,run_best_so_far_mean,run_best_so_far_std,best_n_feasible_runs,run_best_so_far_stderr,best_t_crit,best_ci_upper,best_ci_lower
i32,f64,f64,u32,f64,f64,f64,f64
1,null,null,0,null,null,null,null
2,-124.410033,2.745495,28,0.51885,2.051831,-123.345441,-125.474625
3,-152.234316,30.237427,62,3.840157,1.999624,-144.555447,-159.913185
4,-170.98481,25.379228,155,2.038509,1.975488,-166.95776,-175.011859
5,-171.169995,24.768678,166,1.922422,1.974446,-167.374277,-174.965713
…,…,…,…,…,…,…,…
96,-242.803676,31.313039,1000,0.990205,1.962341,-240.860555,-244.746797
97,-242.813431,31.314921,1000,0.990265,1.962341,-240.870193,-244.756668
98,-242.840841,31.275615,1000,0.989022,1.962341,-240.900042,-244.781639


## Hypervolume stuff

In [372]:
# Take solutions that have their constraint value between [0, relaxed]
relaxed_con_value = int(ct.split("p")[0]) + float(ct.split("p")[1]) * 10 ** (-len(ct.split("p")[1]))

print(relaxed_con_value)

reference_front = df_front.filter((pl.col("c_1") >= 0) & (pl.col("c_1") <= relaxed_con_value))

f_lo, c_lo = reference_front["f_1_min"].min(), reference_front["c_1_min"].min()
f_hi, c_hi = reference_front["f_1_min"].max(), reference_front["c_1_min"].max()

5.0


In [373]:
mask = pl.col("f_1_min").is_between(f_lo, f_hi, closed="both") & pl.col("c_1").is_between(c_lo, c_hi, closed="both")
filtered = (
    df.group_by(["run", "generation"])
    .agg(
        pl.struct(
            [
                pl.col("f_1_min").alias("f"),
                pl.col("c_1").alias("c"),
            ]
        )
        .filter(mask)
        .alias("shadow_front"),
        pl.struct(
            [
                ((pl.col("f_1_min") - f_lo) / (f_hi - f_lo)).clip(0.0, 1.0).alias("f_norm"),
                ((pl.col("c_1") - c_lo) / (c_hi - c_lo)).clip(0.0, 1.0).alias("c_norm"),
            ]
        )
        .filter(mask)
        .alias("shadow_front_norm"),
    )
    .sort(["run", "generation"])
)

filtered = filtered.with_columns(
    pl.col("shadow_front_norm").map_elements(hv_from_front, return_dtype=pl.Float64).alias("hv")
)

hv_summary = (
    filtered.group_by("generation")
    .agg(
        pl.col("hv").mean().alias("hv_mean"),
        pl.col("hv").std().alias("hv_std"),
        pl.col("hv").count().alias("hv_n_supporting_runs"),
    )
    .with_columns(
        pl.when(pl.col("hv_n_supporting_runs") > 1)
        .then(pl.col("hv_std") / pl.col("hv_n_supporting_runs").sqrt())
        .otherwise(None)
        .alias("hv_stderr")
    )
    .sort("generation")
)

hv_summary = hv_summary.with_columns(
    pl.when(pl.col("hv_n_supporting_runs") > 1)
    .then(
        pl.Series(
            "hv_t_crit",
            t.ppf(0.975, hv_summary["hv_n_supporting_runs"] - 1),
        )
    )
    .otherwise(None)
)

hv_summary = hv_summary.with_columns(
    (pl.col("hv_mean") + pl.col("hv_t_crit") * pl.col("hv_stderr")).alias("hv_ci_upper"),
    (pl.col("hv_mean") - pl.col("hv_t_crit") * pl.col("hv_stderr")).alias("hv_ci_lower"),
)

hv_summary

generation,hv_mean,hv_std,hv_n_supporting_runs,hv_stderr,hv_t_crit,hv_ci_upper,hv_ci_lower
i32,f64,f64,u32,f64,f64,f64,f64
1,null,null,0,null,null,null,null
2,null,null,0,null,null,null,null
3,null,null,0,null,null,null,null
4,null,null,0,null,null,null,null
5,null,null,0,null,null,null,null
…,…,…,…,…,…,…,…
96,0.356854,0.08876,586,0.003667,1.964027,0.364055,0.349653
97,0.402234,0.081667,587,0.003371,1.96402,0.408854,0.395614
98,0.314581,0.102063,585,0.00422,1.964034,0.322869,0.306293


In [374]:
best_hv = reference_front.with_columns(
    pl.struct(
        ((pl.col("f_1_min") - f_lo) / (f_hi - f_lo)).clip(0.0, 1.0).alias("f_norm"),
        ((pl.col("c_1") - c_lo) / (c_hi - c_lo)).clip(0.0, 1.0).alias("c_norm"),
    ).alias("norm"),
)

hv(best_hv["norm"].to_numpy(), 1.0)

0.629028324646058

In [377]:
summary_all = summary.join(hv_summary, on="generation")

summary_all

df_front
df

x_1,x_2,f_1,f_1_min,c_1,generation,run
f64,f64,f64,f64,f64,i32,i32
8.06511,3.902804,-126.891561,-126.891561,8.444131,1,0
-4.646495,0.75658,-417.394825,-417.394825,242.379461,1,0
7.264557,12.560944,-13.431644,-13.431644,138.147231,1,0
0.597151,10.534839,-108.351235,-108.351235,42.524934,1,0
4.679716,8.874035,-65.832868,-65.832868,60.830281,1,0
…,…,…,…,…,…,…
-3.870065,10.497932,-212.647314,-212.647314,10.768464,100,999
-3.239412,10.579177,-194.825699,-194.825699,-0.823038,100,999
-3.474125,9.240304,-214.726151,-214.726151,10.732061,100,999
